# Embarrassment labeling

## Import Library

In [1]:
import os
import pickle

import pandas as pd
import re

from konlpy.tag import Komoran, Okt

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

from tqdm import tqdm

## Hyper_parameters

In [2]:
# Colab
# DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/miso'

# Local
DATA_DIR = '/Users/inseoklee/Desktop/miso/data'
SAVE_DIR = '/Users/inseoklee/Desktop/miso/data'
LOAD_DIR = '/Users/inseoklee/Desktop/miso/model/CNN'

# MAX_LEN -> 패딩 단계에서 설정

## Load Data

In [3]:
df_embarrassed = pd.read_csv(DATA_DIR + '/label_embarrassed.csv')

In [4]:
df_embarrassed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13630 entries, 0 to 13629
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    13630 non-null  object
 1   label   13630 non-null  object
dtypes: object(2)
memory usage: 213.1+ KB


## Tokenize

In [5]:
def preprocessing(text, okt, remove_stopwords=False, stop_words=[]):
    # 함수의 인자는 다음과 같다.
    # text: 전처리할 텍스트
    # okt: okt객체를 반복적으로 생성하지 않고 미리 생성한 후 인자로 받는다.
    # remove_stopwords: 불용어를 제거할지 여부 선택. 기본값은 False
    # stop_words: 불용어 사전은 사용자가 직접 입력.

    # 1. 한글 및 공백을 제외한 문자를 모두 제거
    text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", text)

    # 2. okt 객체를 활용하여 형태소 단위로 나눈다
    word_text = okt.morphs(text, stem=True)

    if remove_stopwords:
        word_text = [token for token in word_text if (not token in stop_words)] #and len(token) > 1]

    return word_text

In [6]:
stop_words = set(['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한'])
okt = Okt()

[0.003s][warning][os,thread] Attempt to protect stack guard pages failed (0x00000001693d4000-0x00000001693e0000).
[0.003s][warning][os,thread] Attempt to deallocate stack guard pages failed.


## Load Model and Tokenizer

In [7]:
def load_tokenizer(path):
    with open(path, 'rb') as f:
        tokenizer = pickle.load(f)
    return tokenizer

model_name = 'trained_model.h5'
tokenizer_name = 'tokenizer.pickle'
model_path = os.path.join(LOAD_DIR, model_name)
tokenizer_path = os.path.join(LOAD_DIR, tokenizer_name)

model = load_model(model_path)
tokenizer = load_tokenizer(tokenizer_path)

Metal device set to: Apple M1


2022-07-13 16:34:11.395112: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-13 16:34:11.395198: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Labeling

In [8]:
def getResult(predict):
    highest = 0
    highestIdx = 0
    for x in range (0,len(predict)) :
        if(predict[x] > highest) :
            highestIdx = x
            highest =predict[x]
    return highestIdx

angry
sad
fear
disgusting
neutral
happy
surprised

In [9]:
def predict_sentiment(text, model):
    tokens = []
    if type(text) == str:
        tokens.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    else:
        tokens.append([]) #string이 아니면 비어있는 값 추가
    token_sequences = tokenizer.texts_to_sequences(tokens)
    if(len(token_sequences) == 0) :
        return "너무 짧아"
    padded_sequences = pad_sequences(token_sequences, maxlen=15)
    predict = model.predict(padded_sequences, verbose=0)
    # print(predict)
    result = getResult(predict[0])
    if result == 0 :
        return 'angry'
    elif result == 1 :
        return 'sad'
    elif result == 2 :
        return 'fear'
    elif result == 3 :
        return 'disgusting'
    elif result == 4 :
        return 'neutral'
    elif result == 5 :
        return 'happy'
    elif result == 6 :
        return 'surprised'

In [10]:
label_embarrassed = []

for idx in tqdm(range(len(df_embarrassed['text'])), desc="Labeling", mininterval=0.01):
    label = predict_sentiment(df_embarrassed['text'][idx], model)
    label_embarrassed.append(label)

label_embarrassed

Labeling:   0%|          | 0/13630 [00:00<?, ?it/s]2022-07-13 16:34:13.321257: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-13 16:34:13.360509: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Labeling: 100%|██████████| 13630/13630 [05:52<00:00, 38.68it/s]


['disgusting',
 'angry',
 'sad',
 'sad',
 'happy',
 'sad',
 'sad',
 'sad',
 'angry',
 'angry',
 'fear',
 'sad',
 'sad',
 'angry',
 'angry',
 'angry',
 'sad',
 'angry',
 'fear',
 'happy',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'fear',
 'sad',
 'sad',
 'sad',
 'angry',
 'sad',
 'sad',
 'sad',
 'sad',
 'angry',
 'sad',
 'sad',
 'sad',
 'angry',
 'neutral',
 'angry',
 'angry',
 'angry',
 'sad',
 'angry',
 'sad',
 'sad',
 'neutral',
 'angry',
 'happy',
 'angry',
 'happy',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'sad',
 'neutral',
 'sad',
 'sad',
 'sad',
 'angry',
 'happy',
 'neutral',
 'sad',
 'happy',
 'sad',
 'sad',
 'sad',
 'angry',
 'neutral',
 'sad',
 'angry',
 'angry',
 'fear',
 'angry',
 'sad',
 'happy',
 'angry',
 'sad',
 'angry',
 'angry',
 'sad',
 'sad',
 'angry',
 'sad',
 'disgusting',
 'sad',
 'angry',
 'sad',
 'sad',
 'angry',
 'angry',
 'disgusting',
 'disgusting',
 'angry',
 'sad',
 'neutral',
 'angry',
 'sad',
 'fear',
 'sad',
 'sad',
 'angry',
 'sad',
 'sad',
 'a

In [11]:
df_embarrassed['label'] = label_embarrassed
df_embarrassed[:20]

,text,label
0,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,disgusting
1,오늘 선생님이 나보고 앞에 나와 문제를 풀어보라고 했는데 답을 외워서 문제를 풀었어.,angry
2,내가 무능해서 자기가 좋은 대학에 못 들어갔다는 딸의 말을 듣고 괴로워.,sad
3,요즘 아들이 말도 없이 사업을 시작했나봐.,sad
4,교수님께서 과제로 내주신 에세이에 엄청 애쓴 내가 너무 한심해.,happy
5,남편이 내 돈을 들고 집을 나갔어. 어떡하지?,sad
6,젊은 시절에 내가 했던 일이 이제 와 한심해.,sad
7,엄마가 신문을 보시다가 성 소수자를 차별하는 발언을 했는데 그 모습이 너무 혐오스러웠어.,sad
8,며칠 전에 친구 집에서 가족들이 모여 함께 텔레비전을 시청하는데 혐오스러운 장면이 ...,angry
9,어제 회사에서 정말 창피한 일을 경험했어.,angry


## Re labeling

### '행복'으로 레이블링된 샘플의 레이블을 '놀람'으로 레이블링

angry
sad
fear
disgusting
neutral
happy
surprised

In [12]:
df_embarrassed.loc[df_embarrassed['label'] == "happiness", 'label'] = 'surprise'

In [13]:
df_embarrassed['label'].unique()

array(['disgusting', 'angry', 'sad', 'happy', 'fear', 'neutral',
       'surprised'], dtype=object)

## Save data

In [14]:
df_embarrassed.to_csv(SAVE_DIR+'/relabel_embarrassed.csv', encoding='utf-8-sig', index=False)